In [1]:
import numpy as np
import astropy
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.table import Table,vstack, Column
import wget
import astropy.units as u
from numpy.ma import masked
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

In [ ]:
url=f'https://s3.amazonaws.com/grizli-v2/JwstMosaics/v7/abell2744clu-grizli-v7.0_phot.fits'
filename=wget.download(url)

# DAWN Comparison Only


In [ ]:
#photometric catalogues
hdul= fits.open('gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abellclu= fits.open('abell2744clu-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gdn = Table.read(hdul)
abell = Table.read(abellclu)


In [2]:
#eazy catalogues
gdn_e= fits.open('MPhys/eazy_comp/gdn-grizli-v7.3-fix.eazypy.zout.fits', memmap=True,formats='fits')
abell_e= fits.open('MPhys/eazy_comp/abell2744clu-grizli-v7.2-fix.eazypy.zout.fits', memmap=True,formats='fits')
gdn_eazy = Table.read(gdn_e)
abell_eazy = Table.read(abell_e)


In [ ]:
big_cat = Table.read('MPhys/final_skymatched_abmag_catalogue.fits',format='fits')

In [ ]:
gdn_checker=np.zeros(len(big_cat))
for i in range(len(big_cat)):
    if big_cat['field'][i] =='gdn':
        gdn_checker[i]=1
gdn_specz = big_cat[np.array(gdn_checker,dtype=bool)]
gdn_specz['z_phot'] = np.zeros(len(gdn_specz))       

In [ ]:
abell_checker=np.zeros(len(big_cat))
for i in range(len(big_cat)):
    if big_cat['field'][i] =='abells2744clu':
        abell_checker[i]=1
abell_specz = big_cat[np.array(abell_checker,dtype=bool)]
abell_specz['z_phot'] = np.zeros(len(abell_specz))       

In [ ]:
gdn_eazy['z_phot'][gdn_eazy['z_phot'] <= 0] = np.nan
cat=np.ones(len(gdn_eazy))
for i in range(len(gdn_eazy)):
    if np.isnan(gdn_eazy['z_phot'][i]):
        cat[i]=0
cat_good = np.array(cat,dtype=bool)
gdn_eazy=gdn_eazy[cat_good]

In [ ]:
gdn_eazy

### Skymatching

In [ ]:
#matching jwst and candels
ra1=gdn['ra']

dec1=gdn['dec']
ra2=gdn_eazy['ra']
dec2=gdn_eazy['dec']


In [ ]:
gdn_cat = SkyCoord(ra=ra1, dec=dec1)#dawn
eazy_cat = SkyCoord(ra=ra2, dec=dec2)#candels


In [ ]:
max_sep = 0.1 * u.arcsec
idx, d2d, d3d = gdn_cat.match_to_catalog_sky(eazy_cat)#catalog = candels information
idx_gdn, d2d_gdn, d3d_gdn = eazy_cat.match_to_catalog_sky(gdn_cat)


In [ ]:
c_index=[]
j_index=[]
for i in range(len(eazy_cat)):
    if abs(ra2[i]-ra1[idx_gdn[i]])<0.1:
        c_index.append(i)
        j_index.append(idx_gdn[i])
        gdn['z_phot'][idx_gdn[i]]=gdn_eazy['z_phot'][i]


# DAWN+EXCELS Comparison